In [2]:
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')

X = mnist["data"]
y = mnist["target"]

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [3]:
import numpy as np
np.random.seed(42)
rnd_idx = np.random.permutation(60000)
X_train = X_train[rnd_idx]
y_train = y_train[rnd_idx]

In [4]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [6]:
lin_clf = LinearSVC(random_state=42)

In [7]:
svm_clf = SVC(decision_function_shape="ovr", gamma="auto")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC(gamma='auto')

In [8]:
from sklearn.metrics import accuracy_score
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9476

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ...C=8.852316058423087, gamma=0.0017660746504810706; total time=   0.1s
[CV] END ...C=8.852316058423087, gamma=0.0017660746504810706; total time=   0.1s
[CV] END ...C=8.852316058423087, gamma=0.0017660746504810706; total time=   0.1s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453387; total time=   0.1s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453387; total time=   0.1s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453387; total time=   0.1s
[CV] END .....C=9.875199193765326, gamma=0.05134983345187068; total time=   0.1s
[CV] END .....C=9.875199193765326, gamma=0.05134983345187068; total time=   0.1s
[CV] END .....C=9.875199193765326, gamma=0.05134983345187068; total time=   0.1s
[CV] END .....C=6.59992909281409, gamma=0.059916665784661816; total time=   0.1s
[CV] END .....C=6.59992909281409, gamma=0.059916665784661816; total time=   0.1s
[CV] END .....C=6.59992909281409, gamma=0.059916

RandomizedSearchCV(cv=3, estimator=SVC(gamma='auto'),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001E6DDD734A0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001E6DDD73A70>},
                   verbose=2)

In [23]:
rnd_search_cv.best_estimator_

SVC(C=8.852316058423087, gamma=0.0017660746504810706)

In [25]:
rnd_search_cv.best_score_

0.8630037222851593

In [27]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=8.852316058423087, gamma=0.0017660746504810706)

In [30]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.99965

In [32]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.971